In [ ]:
!pip -q install scikit-image scikit-learn pillow

import os, glob, math, random
from dataclasses import dataclass
from typing import Dict, List, Tuple, Optional, Any

import numpy as np
from PIL import Image
from sklearn.cluster import MiniBatchKMeans
from skimage.color import rgb2lab

In [ ]:
# =========================
# (A) 파일/이미지 설정
# =========================
# 필요시 db 형태로 변환해야 함.
IMG_DIR = "/content/images"   # <-- 여기에 image_id로 된 이미지들이 있는 폴더를 둔다고 가정
IMG_EXTS = (".jpg", ".jpeg", ".png", ".webp")

# =========================
# (B) 추천 파라미터 (5~10 권장)
# =========================
K = 7      # part별 top-k (이미 recommend_topk... 호출 시 k로 쓰는 값)
L = 7      # top/bottom 전수조합(k^2)에서 뽑을 상하세트 개수
M = 10     # 최종 outer x (inner) 조합에서 뽑을 개수

# =========================
# (C) 색 추출 파라미터
# =========================
CROP_PARTS = ("상의", "하의")     # 상/하의는 중앙 정사각 크롭
KMEANS_K = 5                     # 주색 후보 클러스터 수
PIXEL_SAMPLE = 6000              # k-means에 넣을 픽셀 샘플 수(속도/안정 밸런스)
RANDOM_STATE = 42

In [ ]:
def resolve_image_path(image_id: str, img_dir: str = IMG_DIR) -> str:
    """
    image_id에 대해 IMG_DIR 내부에서 파일을 찾습니다.
    - {image_id}.jpg/png/... 우선
    - 없으면 glob으로 부분 매칭까지 시도
    """
    image_id = str(image_id)

    for ext in IMG_EXTS:
        p = os.path.join(img_dir, image_id + ext)
        if os.path.exists(p):
            return p

    # fallback: any ext
    cand = []
    for ext in IMG_EXTS:
        cand.extend(glob.glob(os.path.join(img_dir, f"{image_id}*{ext}")))
    if len(cand) == 0:
        raise FileNotFoundError(f"Cannot find image for image_id={image_id} in {img_dir}")
    return cand[0]


def center_square_crop(pil_img: Image.Image) -> Image.Image:
    w, h = pil_img.size
    s = min(w, h)
    left = (w - s) // 2
    top = (h - s) // 2
    return pil_img.crop((left, top, left + s, top + s))


def load_rgb_np(image_path: str, do_center_square_crop: bool) -> np.ndarray:
    img = Image.open(image_path).convert("RGB")
    if do_center_square_crop:
        img = center_square_crop(img)
    return np.asarray(img, dtype=np.uint8)  # (H,W,3) RGB


def dominant_color_lab_kmeans(rgb_uint8: np.ndarray,
                              k: int = KMEANS_K,
                              sample_n: int = PIXEL_SAMPLE,
                              random_state: int = RANDOM_STATE) -> Tuple[np.ndarray, np.ndarray]:
    """
    입력 RGB 이미지에서 LAB 픽셀을 샘플링하고 k-means로 클러스터링한 뒤,
    가장 큰 클러스터의 중심을 '주색'으로 반환.
    returns:
      - dom_lab: (3,) float
      - dom_weight: (1,) float  (주색 클러스터 비중)
    """
    rgb = rgb_uint8.astype(np.float32) / 255.0
    lab = rgb2lab(rgb)  # (H,W,3), L in [0,100], a/b roughly [-128,127]

    H, W, _ = lab.shape
    X = lab.reshape(-1, 3)

    # 샘플링 (속도)
    n = X.shape[0]
    if n > sample_n:
        idx = np.random.default_rng(random_state).choice(n, size=sample_n, replace=False)
        Xs = X[idx]
    else:
        Xs = X

    km = MiniBatchKMeans(
        n_clusters=k,
        random_state=random_state,
        batch_size=2048,
        n_init="auto"
    )
    labels = km.fit_predict(Xs)
    centers = km.cluster_centers_

    # 가장 큰 클러스터 선택
    counts = np.bincount(labels, minlength=k)
    j = int(np.argmax(counts))
    dom = centers[j]
    weight = counts[j] / counts.sum()

    return dom.astype(np.float32), np.array([weight], dtype=np.float32)


@dataclass(frozen=True)
class ItemColor:
    item_id: str
    part: str
    lab: np.ndarray         # (3,)
    weight: float           # 주색 비중(참고용)

In [ ]:
'''
여기서 임의 감성으로 하면 위험하니, 명시적인 규칙 기반 스코어로 구현합니다.
가장 보편적인 규칙은 색상환(hue) 관계 + 명도(L) 대비 + 채도(C) 적정입니다.

LAB → LCh로 변환해서 hue 각도를 사용합니다.

조화는 크게:

보색(≈180°)

유사색(≈30°)

삼각조화(≈120°)
를 선호하도록 가우시안/랩핑 거리로 점수화합니다.

그리고 **명도 대비(outer/inner 등에서 특히 중요)**를 약하게 반영합니다.
'''

def lab_to_lch(lab: np.ndarray) -> Tuple[float, float, float]:
    """
    LAB -> L, C, h(deg)
    """
    L, a, b = float(lab[0]), float(lab[1]), float(lab[2])
    C = math.sqrt(a*a + b*b)
    h = math.degrees(math.atan2(b, a))  # [-180,180]
    if h < 0:
        h += 360.0
    return L, C, h


def angle_diff_deg(h1: float, h2: float) -> float:
    d = abs(h1 - h2) % 360.0
    return min(d, 360.0 - d)


def gaussian_score(x: float, mu: float, sigma: float) -> float:
    # x, mu in degrees or in same unit
    return math.exp(-0.5 * ((x - mu) / sigma) ** 2)


def harmony_score_lab(lab1: np.ndarray, lab2: np.ndarray,
                      w_comp: float = 0.50, w_anal: float = 0.30, w_tria: float = 0.20,
                      sigma_comp: float = 25.0, sigma_anal: float = 18.0, sigma_tria: float = 22.0,
                      w_light: float = 0.25, sigma_light: float = 25.0,
                      w_chroma: float = 0.10) -> float:
    """
    '가까움'이 아니라 '조화' 기반 스코어 (클수록 좋음)

    - hue 관계: 보색/유사/삼각 조화 점수의 가중합
    - 명도 대비: 너무 비슷한 것보다 어느 정도 차이를 선호(outer-inner에 유리)
    - 채도(C): 극단적으로 낮거나 높은 경우 약한 패널티/보정(너무 과한 영향은 배제)
    """
    L1, C1, h1 = lab_to_lch(lab1)
    L2, C2, h2 = lab_to_lch(lab2)

    dh = angle_diff_deg(h1, h2)

    s_comp = gaussian_score(dh, 180.0, sigma_comp)
    s_anal = gaussian_score(dh,  30.0, sigma_anal) + gaussian_score(dh, 0.0, sigma_anal)  # 0~30 근방
    s_tria = gaussian_score(dh, 120.0, sigma_tria)

    hue_harmony = w_comp*s_comp + w_anal*s_anal + w_tria*s_tria

    # 명도 대비: |L1-L2|가 너무 0이면 약하고, 어느 정도 차이가 있으면 유리
    dL = abs(L1 - L2)
    light = 1.0 - gaussian_score(dL, 0.0, sigma_light)  # 0이면 0에 가깝고, 멀어질수록 1에 접근

    # 채도: 너무 낮으면(무채색) hue 의미가 약해지므로 약하게 보정
    # 아주 단순히 평균 채도가 너무 낮으면 전체를 조금 깎고, 적당하면 유지
    Cmean = (C1 + C2) / 2.0
    chroma_boost = 1.0 - math.exp(-Cmean / 25.0)  # Cmean 작으면 0에 가깝고, 커질수록 1

    score = hue_harmony + w_light*light + w_chroma*chroma_boost
    return float(score)

In [ ]:
'''
주피터 파일의 recommend_topk_by_part_with_temp(...) 결과를 그대로 넣으면 됩니다.

res["상의"] 같은 리스트에서 item_id를 꺼내 이미지 로드 → 주색 추출
'''

def extract_colors_from_topk(topk_res: Dict[str, List[dict]],
                             img_dir: str = IMG_DIR) -> Dict[str, List[ItemColor]]:
    """
    topk_res: recommend_topk_by_part_with_temp(...)의 반환값
      { "상의": [{"item_id":..., ...}, ...], "하의": [...], ... }

    returns:
      { "상의": [ItemColor...], ... }
    """
    cache: Dict[str, ItemColor] = {}
    out: Dict[str, List[ItemColor]] = {}

    for part, rows in topk_res.items():
        colors = []
        for r in rows:
            iid = str(r["item_id"])
            if iid in cache:
                colors.append(cache[iid])
                continue

            path = resolve_image_path(iid, img_dir)
            do_crop = (part in CROP_PARTS)
            rgb = load_rgb_np(path, do_center_square_crop=do_crop)
            dom_lab, w = dominant_color_lab_kmeans(rgb, k=KMEANS_K, sample_n=PIXEL_SAMPLE)

            ic = ItemColor(item_id=iid, part=part, lab=dom_lab, weight=float(w[0]))
            cache[iid] = ic
            colors.append(ic)

        out[part] = colors

    return out

In [ ]:
'''
(상의_i, 하의_j) 전부 생성

스코어링

상위 L개 선택

중복 허용: 상의가 여러 세트에 들어가도 OK
'''

@dataclass(frozen=True)
class OutfitSetTB:
    top_id: str
    bottom_id: str
    score: float

def build_top_bottom_sets(top_colors: List[ItemColor],
                          bottom_colors: List[ItemColor],
                          L: int = L) -> List[OutfitSetTB]:
    combos: List[OutfitSetTB] = []
    for t in top_colors:
        for b in bottom_colors:
            s = harmony_score_lab(t.lab, b.lab)
            combos.append(OutfitSetTB(top_id=t.item_id, bottom_id=b.item_id, score=s))

    combos.sort(key=lambda x: x.score, reverse=True)
    return combos[:min(L, len(combos))]

In [ ]:
'''inner 후보 = (원피스 k) + (상하세트 L) → outer k와 전수조합 → Top-M

inner는 두 타입:

원피스: item 하나

상하세트: (top,bottom) 두 개

outer와의 스코어:

원피스: harmony(outer, onepiece)

상하세트: 0.5*harmony(outer, top) + 0.5*harmony(outer, bottom) (기본)

최종적으로 outer x inner 전수조합해서 Top-M 반환

중복 허용: 같은 outer/inner가 여러 결과에 있어도 O'''

@dataclass(frozen=True)
class InnerCandidate:
    kind: str              # "onepiece" or "tbset"
    ids: Tuple[str, ...]   # ("onepiece_id",) or ("top_id","bottom_id")
    score_hint: float      # inner 내부 스코어(원피스는 0, tbset은 상하조화)

@dataclass(frozen=True)
class FinalOutfit:
    outer_id: str
    inner: InnerCandidate
    score: float

def build_inner_candidates(onepiece_colors: List[ItemColor],
                           tb_sets: List[OutfitSetTB]) -> List[InnerCandidate]:
    inners: List[InnerCandidate] = []

    for op in onepiece_colors:
        inners.append(InnerCandidate(kind="onepiece", ids=(op.item_id,), score_hint=0.0))

    for s in tb_sets:
        inners.append(InnerCandidate(kind="tbset", ids=(s.top_id, s.bottom_id), score_hint=float(s.score)))

    return inners


def index_by_id(colors_by_part: Dict[str, List[ItemColor]]) -> Dict[str, ItemColor]:
    m = {}
    for part, items in colors_by_part.items():
        for it in items:
            m[it.item_id] = it
    return m


def outer_inner_score(outer: ItemColor,
                      inner: InnerCandidate,
                      color_index: Dict[str, ItemColor],
                      w_inner_hint: float = 0.10,
                      w_tb_top: float = 0.50,
                      w_tb_bottom: float = 0.50) -> float:
    """
    outer와 inner의 조합 점수
    - 기본은 outer- inner(원피스 또는 상/하 각각) 조화
    - tbset인 경우 상/하 각각과의 조화 평균
    - inner 자체 점수(tb의 top-bottom 조화)도 약간 반영(옵션)
    """
    if inner.kind == "onepiece":
        op = color_index[inner.ids[0]]
        base = harmony_score_lab(outer.lab, op.lab)
    elif inner.kind == "tbset":
        t = color_index[inner.ids[0]]
        b = color_index[inner.ids[1]]
        base = w_tb_top*harmony_score_lab(outer.lab, t.lab) + w_tb_bottom*harmony_score_lab(outer.lab, b.lab)
    else:
        raise ValueError(f"Unknown inner kind: {inner.kind}")

    return float(base + w_inner_hint * inner.score_hint)


def build_final_outfits(outer_colors: List[ItemColor],
                        inner_candidates: List[InnerCandidate],
                        colors_by_part: Dict[str, List[ItemColor]],
                        M: int = M) -> List[FinalOutfit]:
    color_idx = index_by_id(colors_by_part)

    allc: List[FinalOutfit] = []
    for o in outer_colors:
        for inner in inner_candidates:
            s = outer_inner_score(o, inner, color_idx)
            allc.append(FinalOutfit(outer_id=o.item_id, inner=inner, score=s))

    allc.sort(key=lambda x: x.score, reverse=True)
    return allc[:min(M, len(allc))]

In [ ]:
'''실행 예시 (당신의 top-k 결과를 넣기)

당신 주피터 파일에서 이미 아래가 가능하죠:

res = recommend_topk_by_part_with_temp(..., k=K, parts=("상의","하의","아우터","원피스"), ...)

res 형태는 {part: [ {item_id:...}, ... ]}

그걸 그대로 넣으면 됩니다'''
# (1) 먼저 주피터/코랩에서 top-k를 얻었다고 가정:
# res = recommend_topk_by_part_with_temp(...)

# 여기서는 res가 이미 존재한다고 가정하고 진행합니다.
assert "상의" in res and "하의" in res and "원피스" in res and "아우터" in res, "res에 4개 part가 필요합니다."

# (2) top-k 아이템들의 주색 추출
colors_by_part = extract_colors_from_topk(res, img_dir=IMG_DIR)

top_colors = colors_by_part["상의"]
bottom_colors = colors_by_part["하의"]
onepiece_colors = colors_by_part["원피스"]
outer_colors = colors_by_part["아우터"]

print(len(top_colors), len(bottom_colors), len(onepiece_colors), len(outer_colors))

# (3) 상/하 전수조합(k^2) -> Top-L 세트
tb_sets = build_top_bottom_sets(top_colors, bottom_colors, L=L)
print("Top-Bottom sets:", len(tb_sets), " (example)", tb_sets[:3])

# (4) inner 후보 = 원피스 k + 상하세트 L
inner_candidates = build_inner_candidates(onepiece_colors, tb_sets)
print("Inner candidates:", len(inner_candidates))

# (5) outer k와 inner (k+L) 전수조합 -> Top-M 최종
final_outfits = build_final_outfits(outer_colors, inner_candidates, colors_by_part, M=M)

print("\n=== Final outfits (Top-M) ===")
for i, fo in enumerate(final_outfits, 1):
    if fo.inner.kind == "onepiece":
        inner_str = f"onepiece={fo.inner.ids[0]}"
    else:
        inner_str = f"top={fo.inner.ids[0]}, bottom={fo.inner.ids[1]}"
    print(f"{i:>2}. outer={fo.outer_id} | {inner_str} | score={fo.score:.4f}")

In [ ]:
'''시연에서 “깔끔함” 보여주려면 최종 Top-M을 이미지로 띄우는 게 좋습니다.'''

from IPython.display import display

def show_image_by_id(image_id: str, title: str = "", size: int = 240):
    p = resolve_image_path(image_id, IMG_DIR)
    img = Image.open(p).convert("RGB")
    img = img.resize((size, size))
    if title:
        print(title)
    display(img)

def visualize_final_outfits(final_outfits: List[FinalOutfit], max_show: int = 10):
    for i, fo in enumerate(final_outfits[:max_show], 1):
        print(f"\n[{i}] score={fo.score:.4f}")
        show_image_by_id(fo.outer_id, title=f"outer: {fo.outer_id}")
        if fo.inner.kind == "onepiece":
            show_image_by_id(fo.inner.ids[0], title=f"onepiece: {fo.inner.ids[0]}")
        else:
            show_image_by_id(fo.inner.ids[0], title=f"top: {fo.inner.ids[0]}")
            show_image_by_id(fo.inner.ids[1], title=f"bottom: {fo.inner.ids[1]}")

visualize_final_outfits(final_outfits, max_show=M)